<a href="https://colab.research.google.com/github/hanghae-plus-AI/AI-1-csn2506/blob/main/week6/Chapter4_1_%EA%B8%B0%EB%B3%B8%EA%B3%BC%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 4-1- 기본과제

## Validation data를 포함하여 Fine-tuning 해보기

In [ ]:
!pip install datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.4 MB/s eta 0:00:00


In [ ]:
import os

import sys

import math

import torch

import wandb

import logging

import datasets

import argparse

import evaluate

import transformers



from typing import Optional

from itertools import chain

from dataclasses import dataclass, field



from datasets import load_dataset

from transformers import (

    AutoConfig,

    AutoModelForCausalLM,

    AutoTokenizer,

    HfArgumentParser,

    Trainer,

    TrainingArguments,

    default_data_collator

)

from transformers.trainer_utils import get_last_checkpoint

In [ ]:
wandb.init(project='Chapter4-1-basic')

wandb.run.name = 'gpts-finetuning'

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
@dataclass

class Arguments:

    model_name_or_path: Optional[str] = field(default=None)  # HuggingFace hub에서 pre-trained 모델로 사용할 모델의 이름

    torch_dtype: Optional[str] = field(default=None, metadata={'choices': ['auto', 'bfloat16', 'float16', 'float32']})  # 우리 모델의 precision(data type이라고 이해하시면 됩니다)



    dataset_name: Optional[str] = field(default=None)  # Fine-tuning으로 사용할 huggingface hub에서의 dataset 이름

    dataset_config_name: Optional[str] = field(default=None)  # Fine-tuning으로 사용할 huggingface hub에서의 dataset configuration

    block_size: int = field(default=1024)  # Fine-tuning에 사용할 input text의 길이

    num_workers: Optional[int] = field(default=None)  # Data를 업로드하거나 전처리할 때 사용할 worker 숫자



import sys

sys.argv = [

    sys.argv[0],

    "--output_dir", "./output",

    "--per_device_train_batch_size", "8",

    "--per_device_eval_batch_size", "8",

    "--num_train_epochs", "3",

    "--logging_dir", "./logs",

    "--dataset_name", "wikitext",

    "--model_name_or_path", "openai-community/openai-gpt",

    "--dataset_config_name", "wikitext-2-raw-v1",

    "--do_train",

    "--save_total_limit", "1",

    "--logging_steps", "100",

]





parser = HfArgumentParser((Arguments, TrainingArguments))

args, training_args = parser.parse_args_into_dataclasses()



logger = logging.getLogger()



logging.basicConfig(

    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",

    datefmt="%m/%d/%Y %H:%M:%S",

    handlers=[logging.StreamHandler(sys.stdout)],

)



if training_args.should_log:

    transformers.utils.logging.set_verbosity_info()  # log level을 INFO로 변경



log_level = training_args.get_process_log_level()

logger.setLevel(log_level)





# 우리가 가지고 있는 logger와 HuggingFace의 logger의 log level 설정

logger.setLevel(log_level)

datasets.utils.logging.set_verbosity(log_level)

transformers.utils.logging.set_verbosity(log_level)



# 기타 HuggingFace logger option들을 설정

transformers.utils.logging.enable_default_handler()

transformers.utils.logging.enable_explicit_format()



logger.info(f"Training/evaluation parameters {training_args}")

## 데이터셋 로드

In [ ]:




raw_datasets = load_dataset(

    args.dataset_name,

    args.dataset_config_name

)


README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

Generating dataset wikitext (/root/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/0.0.0/b08601e04326c79dfdd32d625aee71d232d685c3)


test-00000-of-00001.parquet:   0%|          | 0.00/733k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/6.36M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/657k [00:00<?, ?B/s]

...

Generating train split


Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split


Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

All the splits matched successfully.
Dataset wikitext downloaded and prepared to /root/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/0.0.0/b08601e04326c79dfdd32d625aee71d232d685c3. Subsequent calls will reuse this data.


In [ ]:
config = AutoConfig.from_pretrained(args.model_name_or_path)

tokenizer = AutoTokenizer.from_pretrained(args.model_name_or_path)

model = AutoModelForCausalLM.from_pretrained(

    args.model_name_or_path,

    config=config,

    torch_dtype=args.torch_dtype

)



tokenizer.pad_token_id = tokenizer.eos_token_id

tokenizer.chat_template = "{% for message in messages %}{{'<|im_start|>' + message['role'] + '\n' + message['content'] + '<|im_end|>' + '\n'}}{% endfor %}"



embedding_size = model.get_input_embeddings().weight.shape[0]

if len(tokenizer) > embedding_size:

    model.resize_token_embeddings(len(tokenizer))



column_names = list(raw_datasets["train"].features)

text_column_name = "text" if "text" in column_names else column_names[0]



def tokenize_function(examples):

    output = tokenizer(examples[text_column_name])

    return output



with training_args.main_process_first(desc="dataset map tokenization"):

    tokenized_datasets = raw_datasets.map(

        tokenize_function,

        batched=True,

        num_proc=args.num_workers,

        remove_columns=column_names

    )



max_pos_embeddings = config.max_position_embeddings if hasattr(config, "max_position_embeddings") else 1024

block_size = args.block_size if tokenizer.model_max_length is None else min(args.block_size, tokenizer.model_max_length)



def group_texts(examples):

    # 주어진 text들을 모두 concat 해줍니다.

    # 예를 들어 examples = {'train': [['Hello!'], ['Yes, that is great!']]}이면 결과물은 {'train': ['Hello! Yes, that is great!']}가 됩니다.

    concatenated_examples = {k: list(chain(*examples[k])) for k in examples.keys()}



    # 전체 길이를 측정합니다.

    total_length = len(concatenated_examples[list(examples.keys())[0]])

    total_length = (total_length // block_size) * block_size



    # block_size로 text를 쪼갭니다.

    # 예를 들어 block_size=3일 때 {'train': ['Hello! Yes, that is great!']}는

    # {'train': ['Hel', 'lo!', ' Ye', 's, ', 'tha', ...]}가 됩니다.

    result = {

        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]

        for k, t in concatenated_examples.items()

    }



    # Next token prediction이니 label은 자기 자신으로 설정합니다.

    result["labels"] = result["input_ids"].copy()

    return result



with training_args.main_process_first(desc="grouping texts together"):

    lm_datasets = tokenized_datasets.map(

        group_texts,

        batched=True,

        num_proc=args.num_workers

    )


config.json:   0%|          | 0.00/656 [00:00<?, ?B/s]

[INFO|configuration_utils.py:672] 2024-10-23 14:07:10,944 >> loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--openai-community--openai-gpt/snapshots/1e0d4f3028acbffb47fe933cea64619c5ec1a002/config.json
[INFO|configuration_utils.py:739] 2024-10-23 14:07:10,948 >> Model config OpenAIGPTConfig {
  "_name_or_path": "openai-community/openai-gpt",
  "afn": "gelu",
  "architectures": [
    "OpenAIGPTLMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "embd_pdrop": 0.1,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "openai-gpt",
  "n_ctx": 512,
  "n_embd": 768,
  "n_head": 12,
  "n_layer": 12,
  "n_positions": 512,
  "n_special": 0,
  "predict_special_tokens": true,
  "resid_pdrop": 0.1,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length"

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

[INFO|configuration_utils.py:672] 2024-10-23 14:07:11,126 >> loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--openai-community--openai-gpt/snapshots/1e0d4f3028acbffb47fe933cea64619c5ec1a002/config.json
[INFO|configuration_utils.py:739] 2024-10-23 14:07:11,128 >> Model config OpenAIGPTConfig {
  "_name_or_path": "openai-community/openai-gpt",
  "afn": "gelu",
  "architectures": [
    "OpenAIGPTLMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "embd_pdrop": 0.1,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "openai-gpt",
  "n_ctx": 512,
  "n_embd": 768,
  "n_head": 12,
  "n_layer": 12,
  "n_positions": 512,
  "n_special": 0,
  "predict_special_tokens": true,
  "resid_pdrop": 0.1,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length"

vocab.json:   0%|          | 0.00/816k [00:00<?, ?B/s]

...

Caching processed dataset at /root/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/0.0.0/b08601e04326c79dfdd32d625aee71d232d685c3/cache-94180eb137c8bd7e.arrow


Map:   0%|          | 0/36718 [00:00<?, ? examples/s]

Caching processed dataset at /root/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/0.0.0/b08601e04326c79dfdd32d625aee71d232d685c3/cache-0b7a9c2b31c919af.arrow


Map:   0%|          | 0/3760 [00:00<?, ? examples/s]

Caching processed dataset at /root/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/0.0.0/b08601e04326c79dfdd32d625aee71d232d685c3/cache-e5e9136e48557ffa.arrow


In [ ]:


# train_dataset = lm_datasets["train"]

train_val_split = lm_datasets["train"].train_test_split(test_size=0.2, seed=42)

train_dataset = train_val_split["train"]

eval_dataset = train_val_split["test"]



trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=default_data_collator
)

Caching indices mapping at /root/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/0.0.0/b08601e04326c79dfdd32d625aee71d232d685c3/cache-d2d6999a00315909.arrow
Caching indices mapping at /root/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/0.0.0/b08601e04326c79dfdd32d625aee71d232d685c3/cache-fd01efde58090c7a.arrow


In [ ]:


checkpoint = None

last_checkpoint = get_last_checkpoint(training_args.output_dir)  # 만약 output_dir에 checkpoint가 남아있으면 이를 사용하고, 없으면 None이 return됩니다.

if training_args.resume_from_checkpoint is not None:  # output_dir이 아닌 다른 위치에서의 checkpoint를 resume_from_checkpoint로 지정할 수 있습니다.

    checkpoint = training_args.resume_from_checkpoint

else:  # 아니면 last_checkpoint로 checkpoint를 지정합니다.

    checkpoint = last_checkpoint


eval_results = trainer.evaluate()
wandb.log({"eval_results": eval_results})

train_result = trainer.train(resume_from_checkpoint=checkpoint)


trainer.save_model()


train_metrics = train_result.metrics

eval_metrics = eval_results



trainer.log_metrics("eval", eval_metrics)

trainer.save_metrics("eval", eval_metrics)



trainer.log_metrics("train", train_metrics)

trainer.save_metrics("train", train_metrics)

trainer.save_state()

wandb.finish()

[INFO|trainer.py:4021] 2024-10-23 14:07:27,209 >> 
***** Running Evaluation *****
[INFO|trainer.py:4023] 2024-10-23 14:07:27,211 >>   Num examples = 917
[INFO|trainer.py:4026] 2024-10-23 14:07:27,212 >>   Batch size = 8


[INFO|integration_utils.py:811] 2024-10-23 14:07:51,912 >> Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
[INFO|trainer.py:2243] 2024-10-23 14:07:52,326 >> ***** Running training *****
[INFO|trainer.py:2244] 2024-10-23 14:07:52,327 >>   Num examples = 3,664
[INFO|trainer.py:2245] 2024-10-23 14:07:52,329 >>   Num Epochs = 3
[INFO|trainer.py:2246] 2024-10-23 14:07:52,330 >>   Instantaneous batch size per device = 8
[INFO|trainer.py:2249] 2024-10-23 14:07:52,332 >>   Total train batch size (w. parallel, distributed & accumulation) = 8
[INFO|trainer.py:2250] 2024-10-23 14:07:52,333 >>   Gradient Accumulation steps = 1
[INFO|trainer.py:2251] 2024-10-23 14:07:52,334 >>   Total optimization steps = 1,374
[INFO|trainer.py:2252] 202

Step,Training Loss
100,4.075800
200,3.931100
300,3.888300
400,3.861000
500,3.747800
600,3.645900
700,3.630600
800,3.637500
900,3.634300
1000,3.523400


[INFO|trainer.py:3705] 2024-10-23 14:12:59,379 >> Saving model checkpoint to ./output/checkpoint-500
[INFO|configuration_utils.py:407] 2024-10-23 14:12:59,382 >> Configuration saved in ./output/checkpoint-500/config.json
[INFO|configuration_utils.py:868] 2024-10-23 14:12:59,384 >> Configuration saved in ./output/checkpoint-500/generation_config.json
[INFO|modeling_utils.py:2836] 2024-10-23 14:13:00,389 >> Model weights saved in ./output/checkpoint-500/model.safetensors
[INFO|tokenization_utils_base.py:2649] 2024-10-23 14:13:00,391 >> tokenizer config file saved in ./output/checkpoint-500/tokenizer_config.json
[INFO|tokenization_utils_base.py:2658] 2024-10-23 14:13:00,393 >> Special tokens file saved in ./output/checkpoint-500/special_tokens_map.json
[INFO|trainer.py:3705] 2024-10-23 14:18:08,466 >> Saving model checkpoint to ./output/checkpoint-1000
[INFO|configuration_utils.py:407] 2024-10-23 14:18:08,470 >> Configuration saved in ./output/checkpoint-1000/config.json
[INFO|configurati

***** eval metrics *****
  eval_loss                   =     4.3906
  eval_model_preparation_time =      0.006
  eval_runtime                = 0:00:25.72
  eval_samples_per_second     =      35.64
  eval_steps_per_second       =       4.47
***** train metrics *****
  epoch                    =        3.0
  total_flos               =  2674823GF
  train_loss               =     3.6916
  train_runtime            = 0:14:10.94
  train_samples_per_second =     12.917
  train_steps_per_second   =      1.615


eval/loss,▁
eval/model_preparation_time,▁
eval/runtime,▁
eval/samples_per_second,▁
eval/steps_per_second,▁
train/epoch,▁▂▂▃▃▄▄▅▅▆▆▇██
train/global_step,▁▁▂▂▃▃▄▄▅▅▆▆▇▇██
train/grad_norm,▅█▄▂▁▂▄▁▄▃▂▃▁
train/learning_rate,█▇▇▆▆▅▅▄▃▃▂▂▁
train/loss,█▆▆▅▄▃▃▃▃▁▁▁▁
eval/loss,4.39061
